# Baba Is You Bot

In [1]:
import pyautogui
import time
from Quartz import CGWindowListCopyWindowInfo, kCGWindowListOptionOnScreenOnly,kCGNullWindowID, kCGWindowListExcludeDesktopElements, CFArrayRef
from PIL import Image
from resizeimage import resizeimage
import math
import os
import numpy as np

In [57]:
screenWidth, screenHeight = pyautogui.size()
# Monitor is either home or laptop
monitor = 'home'

In [50]:
# Getting a list of active windows 
windowList  = CGWindowListCopyWindowInfo(kCGWindowListOptionOnScreenOnly | kCGWindowListExcludeDesktopElements, kCGNullWindowID)

In [51]:
# Getting the baba is you window bounds
for window in windowList:
    if window['kCGWindowName'] == 'Baba Is You':
        windowBounds = window['kCGWindowBounds']

In [91]:
# Getting position and size of Baba window for screenshot
# For some inexplicable reason I have to multiply everything by 2
# I need to subtract 45 in order to not include the top bar
x = windowBounds['X']
y = (windowBounds['Y']) + 45
height = (windowBounds['Height']) - 45 
width = windowBounds['Width']

In [80]:
# Largest window bounds dictionary based on monitor, [height, width]
scalingDict = {'laptop':[805*2,1440*2], 'home':[976*2,1920*2]}

In [92]:
# making array of game state to later insert icons into
gameArray = np.zeros([int(height),int(width)])

In [93]:
# Making a dictionary of icons to ints
iconDict = {"Baba":16, "BabaText":1, "Rock": 2, "RockText": 3, "Flag": 4, "FlagText": 5, \
"IsText": 6, "PushText": 7, "YouText": 8, "WallText": 9, "WinText": 10, "StopText": 11, \
"WaterText": 12, "SinkText": 13, "Wall1": 14, "Wall2": 15, "Water": 15}

In [94]:
# scaling factors for icons (it seems like height changes as a function of width)
# This is fairly hacky but it basically seems to work
scaleW = width/scalingDict[monitor][1]
scaleH = (height/scalingDict[monitor][0]) * scaleW

In [95]:
if monitor =='home':
    path = '/Users/personal/Desktop/BabaIsYou/IconsHome/'
    directory = os.fsencode(path)
else:
    path = '/Users/personal/Desktop/BabaIsYou/IconsLaptop/'
    directory = os.fsencode(path)

In [96]:
def resizeIm(fn):
    with open(path + fn, 'r+b') as f:
        with Image.open(f) as image:
            imW, imH = image.size
            cover = resizeimage.resize_cover(image, [int(math.floor(imH*scaleH)), \
                int(math.floor(imW*scaleW))])
    return(cover, imH*scaleH, imW*scaleW)

In [97]:
# This finds all the instances of an icon within the game and adds them to an array
# It attempts to not duplicate the same object by keeping a 10 pixel threshold around each icon
def findInstances(pics):
    firstPic = pics[0]
    # For now, move to instances of icon to make sure the code is working
    pyautogui.moveTo(firstPic.left/2, firstPic.top/2)
    gameArray[math.ceil(firstPic.top/2),math.ceil(firstPic.left/2)] = int(iconDict[str.split(filename,'.')[0]])
    for pic in pics[1:]:
        if (pic.top >= firstPic.top + 10) or (pic.top <= firstPic.top - 10):
            # Move mouse to instance
            pyautogui.moveTo(pic.left/2, pic.top/2)
            gameArray[math.ceil(pic.top/2),math.ceil(pic.left/2)] = int(iconDict[str.split(filename,'.')[0]])
            firstPic = pic
        else:
            continue

In [98]:
# Creating array of objects from game state
time.sleep(2)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    # Grab the image
    if filename.endswith(".png"):
        print(filename)
        # Rescale
        scaledIcon = resizeIm(filename)[0]
        sizeH, sizeW = resizeIm(filename)[1:]
        print(sizeH,sizeW)
        print(x,y,width,height)
        # Skipping these for now because it thinks the entire screen is an instance
        if filename == 'Wall2.png' or filename == 'Wall1.png' or filename == 'Water.png':
            continue
        # Find in game
        else:
            pyautogui.moveTo(int(x), int(y))
            pics = list(pyautogui.locateAllOnScreen(scaledIcon, confidence = 0.7, \
                region = (int(x), int(y), int(width), int(height)), grayscale=False))
        # Insert into array, if it's in the game
        if pics:
            findInstances(pics)
    else:
        continue

Rock.png
4.301090628201844 9.01796875
1241.0 68.0 679.0 931.0


IndexError: index 816 is out of bounds for axis 1 with size 679

In [31]:
def moveIcon(icon,steps=0):
    time.sleep(2)
    babaLoc = np.argwhere(gameArray==iconDict['Baba'])
    iconLoc = np.argwhere(gameArray==iconDict[icon])
    moves = babaLoc-iconLoc
    # need an if statement to handle whenever there isn't an icon
    move = moves[0,:]
    moveV = np.round_(move[0]/sizeH)
    moveH = np.round_(move[1]/sizeW)
    if moveV != 0.0:
        for i in range(int(np.abs(moveV))+steps):
            time.sleep(1)
            if moveV*(-moveV) == moveV**2:
                pyautogui.press('up')
            else:
                pyautogui.press('down')
    if moveH != 0.0:
        for i in range(int(np.abs(moveH))+steps):
            time.sleep(1)
            if moveH*(-moveH) == moveH**2:
                pyautogui.press('left')
            else:
                print('should go right')
                pyautogui.press('right')

In [32]:
moveIcon('Rock', steps=1)

IndexError: index 0 is out of bounds for axis 0 with size 0